# Topic Resources API Test Cases

This notebook contains comprehensive test cases for the Topic Resources API endpoints.

## Prerequisites
1. Flask server running on `http://localhost:5200`
2. Database table `topic_resources` created in Supabase
3. At least one topic exists in the database

In [18]:
import requests
import json
from datetime import datetime
import pandas as pd

# Configuration
BASE_URL = "https://item-question-service-166647007319.europe-west1.run.app/api"

# Helper function to print formatted JSON
def print_json(data, title="Response"):
    print(f"\n{'='*60}")
    print(f"{title}")
    print('='*60)
    print(json.dumps(data, indent=2))
    print('='*60)

# Test results storage
test_results = []

def log_test(test_name, passed, details=""):
    result = {
        "test_name": test_name,
        "passed": passed,
        "details": details,
        "timestamp": datetime.now().isoformat()
    }
    test_results.append(result)
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"\n{status}: {test_name}")
    if details:
        print(f"Details: {details}")

print("✅ Imports successful!")

✅ Imports successful!


## Setup: Get Test Data

First, we need to get existing hierarchy data to use in our tests.

In [19]:
# Get all exams
response = requests.get(f"{BASE_URL}/hierarchy/exams")
exams = response.json()

if exams and len(exams) > 0:
    test_exam = exams[0]
    exam_id = test_exam['id']
    print(f"Using Exam: {test_exam['name']} (ID: {exam_id})")
else:
    print("⚠️ No exams found. Creating test exam...")
    response = requests.post(f"{BASE_URL}/exams", json={
        "name": "Test Exam",
        "description": "Auto-created for testing"
    })
    test_exam = response.json()[0] if isinstance(response.json(), list) else response.json()
    exam_id = test_exam['id']
    print(f"Created Exam: {test_exam['name']} (ID: {exam_id})")

Using Exam: Notebook School Exam 20251004053309 (ID: edaf8ab2-bcff-4a10-a7f2-005927724f3e)


In [20]:
# Get subjects
if exam_id:
    response = requests.get(f"{BASE_URL}/hierarchy/subjects?exam_id={exam_id}")
    subjects = response.json()
    
    if subjects and len(subjects) > 0:
        test_subject = subjects[0]
        subject_id = test_subject['id']
        print(f"Using Subject: {test_subject['name']} (ID: {subject_id})")
    else:
        print("⚠️ No subjects found. Creating test subject...")
        response = requests.post(f"{BASE_URL}/subjects", json={
            "exam_id": exam_id,
            "name": "Biology",
            "description": "Auto-created for testing"
        })
        test_subject = response.json()[0] if isinstance(response.json(), list) else response.json()
        subject_id = test_subject['id']
        print(f"Created Subject: {test_subject['name']} (ID: {subject_id})")

Using Subject: Test Subject for Resources (ID: faf04c9d-0e58-4a45-b945-a68b799e68fe)


In [21]:
# Get chapters
if subject_id:
    response = requests.get(f"{BASE_URL}/hierarchy/chapters?subject_id={subject_id}")
    chapters = response.json()
    
    if chapters and len(chapters) > 0:
        test_chapter = chapters[0]
        chapter_id = test_chapter['id']
        print(f"Using Chapter: {test_chapter['name']} (ID: {chapter_id})")
    else:
        print("⚠️ No chapters found. Creating test chapter...")
        response = requests.post(f"{BASE_URL}/chapters", json={
            "subject_id": subject_id,
            "name": "Plant Biology",
            "description": "Auto-created for testing"
        })
        test_chapter = response.json()[0] if isinstance(response.json(), list) else response.json()
        chapter_id = test_chapter['id']
        print(f"Created Chapter: {test_chapter['name']} (ID: {chapter_id})")

Using Chapter: Test Chapter for Resources (ID: 8b208535-3682-4e4b-83be-49c644a42b09)


In [6]:
# Get topics
if chapter_id:
    response = requests.get(f"{BASE_URL}/hierarchy/topics?chapter_id={chapter_id}")
    topics = response.json()
    
    if topics and len(topics) > 0:
        test_topic = topics[0]
        topic_id = test_topic['id']
        print(f"Using Topic: {test_topic['name']} (ID: {topic_id})")
    else:
        print("⚠️ No topics found. Creating test topic...")
        response = requests.post(f"{BASE_URL}/topics", json={
            "chapter_id": chapter_id,
            "name": "Photosynthesis",
            "description": "Auto-created for testing"
        })
        test_topic = response.json()[0] if isinstance(response.json(), list) else response.json()
        topic_id = test_topic['id']
        print(f"Created Topic: {test_topic['name']} (ID: {topic_id})")

print(f"\n✅ Setup Complete!")
print(f"Topic ID for testing: {topic_id}")

Using Topic: Test Topic for Resources (ID: 95f24d0b-8723-49ab-80a3-60ff723cb97a)

✅ Setup Complete!
Topic ID for testing: 95f24d0b-8723-49ab-80a3-60ff723cb97a


## Test 1: Add a Single Video Resource to Topic

In [7]:
# Test adding a video resource to a topic
video_resource = {
    "resource_type": "video",
    "title": "Introduction to Photosynthesis",
    "description": "A comprehensive video explaining the process of photosynthesis in plants",
    "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
    "duration": 600,
    "metadata": {
        "platform": "YouTube",
        "quality": "1080p",
        "language": "English",
        "subtitles": ["English", "Spanish"],
        "creator": "Khan Academy"
    },
    "order_index": 0
}

response = requests.post(
    f"{BASE_URL}/topics/{topic_id}/resources",
    json=video_resource
)

if response.status_code == 201:
    video_result = response.json()
    video_resource_id = video_result['id']
    print_json(video_result, "✅ Video Resource Created")
    log_test("Add Video Resource to Topic", True, f"Resource ID: {video_resource_id}")
else:
    print(f"❌ Failed: {response.status_code}")
    print_json(response.json(), "Error Response")
    log_test("Add Video Resource to Topic", False, f"Status: {response.status_code}")
    video_resource_id = None


✅ Video Resource Created
{
  "created_at": "2025-10-04T12:24:28.22541+00:00",
  "description": "A comprehensive video explaining the process of photosynthesis in plants",
  "duration": 600,
  "file_size": null,
  "id": "82e6ac1c-f12a-4206-a4b8-3f2aa08d52ba",
  "is_active": true,
  "metadata": {
    "creator": "Khan Academy",
    "language": "English",
    "platform": "YouTube",
    "quality": "1080p",
    "subtitles": [
      "English",
      "Spanish"
    ]
  },
  "order_index": 0,
  "resource_type": "video",
  "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
  "title": "Introduction to Photosynthesis",
  "topic_id": "95f24d0b-8723-49ab-80a3-60ff723cb97a",
  "updated_at": "2025-10-04T12:24:28.22541+00:00",
  "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
}

✅ PASSED: Add Video Resource to Topic
Details: Resource ID: 82e6ac1c-f12a-4206-a4b8-3f2aa08d52ba


## Test 2: Add 3D Model and Virtual Lab Resources

In [8]:
# Test adding a 3D model resource
model_resource = {
    "resource_type": "3d_model",
    "title": "Chloroplast 3D Model",
    "description": "Interactive 3D model of a chloroplast showing internal structures",
    "url": "https://example.com/models/chloroplast.gltf",
    "thumbnail_url": "https://example.com/thumbnails/chloroplast.png",
    "file_size": 5242880,
    "metadata": {
        "format": "GLTF",
        "polygons": 50000,
        "animated": True,
        "textures": True
    },
    "order_index": 1
}

response = requests.post(
    f"{BASE_URL}/topics/{topic_id}/resources",
    json=model_resource
)

model_resource_id = None
if response.status_code == 201:
    model_result = response.json()
    model_resource_id = model_result['id']
    print_json(model_result, "✅ 3D Model Resource Created")
    log_test("Add 3D Model Resource", True, f"Resource ID: {model_resource_id}")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Add 3D Model Resource", False, f"Status: {response.status_code}")

# Test adding virtual lab
lab_resource = {
    "resource_type": "virtual_lab",
    "title": "Photosynthesis Lab Simulator",
    "description": "Interactive virtual lab to experiment with photosynthesis variables",
    "url": "https://phet.colorado.edu/en/simulation/photosynthesis",
    "metadata": {
        "platform": "PhET",
        "interactive": True,
        "experiments": ["light_intensity", "co2_levels", "temperature"]
    },
    "order_index": 2
}

response = requests.post(
    f"{BASE_URL}/topics/{topic_id}/resources",
    json=lab_resource
)

lab_resource_id = None
if response.status_code == 201:
    lab_result = response.json()
    lab_resource_id = lab_result['id']
    print_json(lab_result, "✅ Virtual Lab Resource Created")
    log_test("Add Virtual Lab Resource", True, f"Resource ID: {lab_resource_id}")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Add Virtual Lab Resource", False, f"Status: {response.status_code}")


✅ 3D Model Resource Created
{
  "created_at": "2025-10-04T12:24:35.26582+00:00",
  "description": "Interactive 3D model of a chloroplast showing internal structures",
  "duration": null,
  "file_size": 5242880,
  "id": "16322da2-152a-4911-a21d-5bfe7f68cfcb",
  "is_active": true,
  "metadata": {
    "animated": true,
    "format": "GLTF",
    "polygons": 50000,
    "textures": true
  },
  "order_index": 1,
  "resource_type": "3d_model",
  "thumbnail_url": "https://example.com/thumbnails/chloroplast.png",
  "title": "Chloroplast 3D Model",
  "topic_id": "95f24d0b-8723-49ab-80a3-60ff723cb97a",
  "updated_at": "2025-10-04T12:24:35.26582+00:00",
  "url": "https://example.com/models/chloroplast.gltf"
}

✅ PASSED: Add 3D Model Resource
Details: Resource ID: 16322da2-152a-4911-a21d-5bfe7f68cfcb

✅ Virtual Lab Resource Created
{
  "created_at": "2025-10-04T12:24:35.746619+00:00",
  "description": "Interactive virtual lab to experiment with photosynthesis variables",
  "duration": null,
  "file

## Test 3: Bulk Add Multiple Resources

In [9]:
# Test bulk adding resources
bulk_resources = {
    "resources": [
        {
            "resource_type": "pdf",
            "title": "Photosynthesis Study Guide",
            "description": "Comprehensive PDF guide",
            "url": "https://example.com/docs/photosynthesis-guide.pdf",
            "file_size": 2048000,
            "metadata": {"pages": 25, "language": "English"}
        },
        {
            "resource_type": "animation",
            "title": "Calvin Cycle Animation",
            "url": "https://example.com/animations/calvin-cycle.mp4",
            "duration": 120
        },
        {
            "resource_type": "article",
            "title": "Recent Research on Photosynthesis",
            "url": "https://example.com/articles/photosynthesis-research",
            "metadata": {"author": "Dr. Smith", "year": 2024}
        },
        {
            "resource_type": "simulation",
            "title": "Light Reactions Simulation",
            "url": "https://example.com/sims/light-reactions",
            "metadata": {"interactive": True}
        }
    ]
}

response = requests.post(
    f"{BASE_URL}/topics/{topic_id}/resources/bulk",
    json=bulk_resources
)

if response.status_code == 201:
    bulk_result = response.json()
    print_json(bulk_result, "✅ Bulk Resources Created")
    log_test("Bulk Add Resources", True, f"Created {bulk_result['created_count']} resources")
else:
    print(f"❌ Failed: {response.status_code}")
    print_json(response.json(), "Error Response")
    log_test("Bulk Add Resources", False, f"Status: {response.status_code}")


✅ Bulk Resources Created
{
  "created_count": 4,
  "resources": [
    {
      "created_at": "2025-10-04T12:24:41.05483+00:00",
      "description": "Comprehensive PDF guide",
      "duration": null,
      "file_size": 2048000,
      "id": "c3bdc0b8-88fe-4560-9a25-fb48d7118099",
      "is_active": true,
      "metadata": {
        "language": "English",
        "pages": 25
      },
      "order_index": 0,
      "resource_type": "pdf",
      "thumbnail_url": null,
      "title": "Photosynthesis Study Guide",
      "topic_id": "95f24d0b-8723-49ab-80a3-60ff723cb97a",
      "updated_at": "2025-10-04T12:24:41.05483+00:00",
      "url": "https://example.com/docs/photosynthesis-guide.pdf"
    },
    {
      "created_at": "2025-10-04T12:24:41.05483+00:00",
      "description": "",
      "duration": 120,
      "file_size": null,
      "id": "902e90ed-c37b-4511-a1b3-e3bea288169e",
      "is_active": true,
      "metadata": {},
      "order_index": 1,
      "resource_type": "animation",
      "th

## Test 4: Get All Resources for Topic

In [10]:
# Get all resources for the topic
response = requests.get(f"{BASE_URL}/topics/{topic_id}/resources")

if response.status_code == 200:
    resources_result = response.json()
    print_json(resources_result, "✅ All Resources for Topic")
    log_test("Get All Topic Resources", True, f"Found {resources_result['count']} resources")
    
    # Display as DataFrame
    if resources_result['resources']:
        df = pd.DataFrame(resources_result['resources'])
        print("\n📊 Resources Summary:")
        print(df[['resource_type', 'title', 'order_index', 'is_active']].to_string())
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Get All Topic Resources", False, f"Status: {response.status_code}")


✅ All Resources for Topic
{
  "count": 7,
  "resources": [
    {
      "created_at": "2025-10-04T12:24:41.05483+00:00",
      "description": "Comprehensive PDF guide",
      "duration": null,
      "file_size": 2048000,
      "id": "c3bdc0b8-88fe-4560-9a25-fb48d7118099",
      "is_active": true,
      "metadata": {
        "language": "English",
        "pages": 25
      },
      "order_index": 0,
      "resource_type": "pdf",
      "thumbnail_url": null,
      "title": "Photosynthesis Study Guide",
      "topic_id": "95f24d0b-8723-49ab-80a3-60ff723cb97a",
      "updated_at": "2025-10-04T12:24:41.05483+00:00",
      "url": "https://example.com/docs/photosynthesis-guide.pdf"
    },
    {
      "created_at": "2025-10-04T12:24:28.22541+00:00",
      "description": "A comprehensive video explaining the process of photosynthesis in plants",
      "duration": 600,
      "file_size": null,
      "id": "82e6ac1c-f12a-4206-a4b8-3f2aa08d52ba",
      "is_active": true,
      "metadata": {
      

## Test 5: Filter Resources by Type

In [11]:
# Get only video resources
response = requests.get(f"{BASE_URL}/topics/{topic_id}/resources?resource_type=video")

if response.status_code == 200:
    videos = response.json()
    print_json(videos, "✅ Video Resources Only")
    log_test("Filter by Video Type", True, f"Found {videos['count']} videos")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Filter by Video Type", False, f"Status: {response.status_code}")

# Get only virtual lab resources
response = requests.get(f"{BASE_URL}/topics/{topic_id}/resources?resource_type=virtual_lab")

if response.status_code == 200:
    labs = response.json()
    print_json(labs, "✅ Virtual Lab Resources Only")
    log_test("Filter by Virtual Lab Type", True, f"Found {labs['count']} labs")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Filter by Virtual Lab Type", False, f"Status: {response.status_code}")


✅ Video Resources Only
{
  "count": 1,
  "resources": [
    {
      "created_at": "2025-10-04T12:24:28.22541+00:00",
      "description": "A comprehensive video explaining the process of photosynthesis in plants",
      "duration": 600,
      "file_size": null,
      "id": "82e6ac1c-f12a-4206-a4b8-3f2aa08d52ba",
      "is_active": true,
      "metadata": {
        "creator": "Khan Academy",
        "language": "English",
        "platform": "YouTube",
        "quality": "1080p",
        "subtitles": [
          "English",
          "Spanish"
        ]
      },
      "order_index": 0,
      "resource_type": "video",
      "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
      "title": "Introduction to Photosynthesis",
      "topic_id": "95f24d0b-8723-49ab-80a3-60ff723cb97a",
      "updated_at": "2025-10-04T12:24:28.22541+00:00",
      "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
    }
  ],
  "topic_id": "95f24d0b-8723-49ab-80a3-60ff723cb97a",
  "topi

## Test 6: Update a Resource

In [12]:
# Update resource
if video_resource_id:
    update_data = {
        "title": "Introduction to Photosynthesis - UPDATED",
        "description": "Updated with enhanced content and examples",
        "duration": 720,
        "metadata": {
            "platform": "YouTube",
            "quality": "4K",
            "language": "English",
            "subtitles": ["English", "Spanish", "French"],
            "updated": True
        }
    }
    
    response = requests.put(
        f"{BASE_URL}/resources/{video_resource_id}",
        json=update_data
    )
    
    if response.status_code == 200:
        updated_resource = response.json()
        print_json(updated_resource, "✅ Resource Updated")
        log_test("Update Resource", True, f"Updated: {updated_resource['title']}")
    else:
        print(f"❌ Failed: {response.status_code}")
        log_test("Update Resource", False, f"Status: {response.status_code}")
else:
    print("⚠️ Skipping: No resource ID available")
    log_test("Update Resource", False, "No resource ID")


✅ Resource Updated
{
  "created_at": "2025-10-04T12:24:28.22541+00:00",
  "description": "Updated with enhanced content and examples",
  "duration": 720,
  "file_size": null,
  "id": "82e6ac1c-f12a-4206-a4b8-3f2aa08d52ba",
  "is_active": true,
  "metadata": {
    "language": "English",
    "platform": "YouTube",
    "quality": "4K",
    "subtitles": [
      "English",
      "Spanish",
      "French"
    ],
    "updated": true
  },
  "order_index": 0,
  "resource_type": "video",
  "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
  "title": "Introduction to Photosynthesis - UPDATED",
  "topic_id": "95f24d0b-8723-49ab-80a3-60ff723cb97a",
  "updated_at": "2025-10-04T12:24:56.988032+00:00",
  "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
}

✅ PASSED: Update Resource
Details: Updated: Introduction to Photosynthesis - UPDATED


## Test 7: Get Chapter Resources (All Topics)

In [13]:
# Get all resources for the chapter (grouped by topics)
response = requests.get(f"{BASE_URL}/chapters/{chapter_id}/resources")

if response.status_code == 200:
    chapter_resources = response.json()
    print_json(chapter_resources, "✅ Chapter Resources (Grouped by Topics)")
    log_test("Get Chapter Resources", True, f"Total: {chapter_resources['total_resources']} resources")
    
    # Display summary
    print("\n📊 Chapter Resources Summary:")
    for topic_data in chapter_resources['topics']:
        print(f"\nTopic: {topic_data['topic_name']}")
        print(f"Resource Count: {topic_data['resource_count']}")
        if topic_data['resources']:
            for res in topic_data['resources']:
                print(f"  - [{res['resource_type']}] {res['title']}")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Get Chapter Resources", False, f"Status: {response.status_code}")


✅ Chapter Resources (Grouped by Topics)
{
  "chapter_id": "8b208535-3682-4e4b-83be-49c644a42b09",
  "chapter_name": "Test Chapter for Resources",
  "topics": [
    {
      "resource_count": 7,
      "resources": [
        {
          "created_at": "2025-10-04T12:24:41.05483+00:00",
          "description": "Comprehensive PDF guide",
          "duration": null,
          "file_size": 2048000,
          "id": "c3bdc0b8-88fe-4560-9a25-fb48d7118099",
          "is_active": true,
          "metadata": {
            "language": "English",
            "pages": 25
          },
          "order_index": 0,
          "resource_type": "pdf",
          "thumbnail_url": null,
          "title": "Photosynthesis Study Guide",
          "topic_id": "95f24d0b-8723-49ab-80a3-60ff723cb97a",
          "updated_at": "2025-10-04T12:24:41.05483+00:00",
          "url": "https://example.com/docs/photosynthesis-guide.pdf"
        },
        {
          "created_at": "2025-10-04T12:24:28.22541+00:00",
         

## Test 8: Soft Delete a Resource

In [14]:
# Soft delete
if model_resource_id:
    response = requests.delete(f"{BASE_URL}/resources/{model_resource_id}")
    
    if response.status_code == 200:
        delete_result = response.json()
        print_json(delete_result, "✅ Resource Soft Deleted")
        log_test("Soft Delete Resource", True, "Resource marked as inactive")
        
        # Verify it's not in active resources
        verify_response = requests.get(f"{BASE_URL}/topics/{topic_id}/resources?is_active=true")
        active_resources = verify_response.json()
        
        is_hidden = all(r['id'] != model_resource_id for r in active_resources['resources'])
        if is_hidden:
            print("✅ Verified: Soft-deleted resource is hidden from active list")
    else:
        print(f"❌ Failed: {response.status_code}")
        log_test("Soft Delete Resource", False, f"Status: {response.status_code}")
else:
    print("⚠️ Skipping: No resource ID available")
    log_test("Soft Delete Resource", False, "No resource ID")


✅ Resource Soft Deleted
{
  "message": "Resource deleted successfully",
  "success": true
}

✅ PASSED: Soft Delete Resource
Details: Resource marked as inactive
✅ Verified: Soft-deleted resource is hidden from active list


## Test 9: Error Handling Tests

In [15]:
# Test invalid resource type
invalid_resource = {
    "resource_type": "invalid_type",
    "title": "Test",
    "url": "https://example.com"
}

response = requests.post(
    f"{BASE_URL}/topics/{topic_id}/resources",
    json=invalid_resource
)

if response.status_code == 400:
    error_result = response.json()
    print_json(error_result, "✅ Correct Error for Invalid Type")
    log_test("Invalid Resource Type Error", True, "Properly rejected")
else:
    log_test("Invalid Resource Type Error", False, f"Got {response.status_code}")

# Test missing required fields
incomplete_resource = {"resource_type": "video"}

response = requests.post(
    f"{BASE_URL}/topics/{topic_id}/resources",
    json=incomplete_resource
)

if response.status_code == 400:
    error_result = response.json()
    print_json(error_result, "✅ Correct Error for Missing Fields")
    log_test("Missing Fields Error", True, "Properly rejected")
else:
    log_test("Missing Fields Error", False, f"Got {response.status_code}")

# Test invalid topic ID
fake_topic_id = "00000000-0000-0000-0000-000000000000"
valid_resource = {
    "resource_type": "video",
    "title": "Test",
    "url": "https://example.com"
}

response = requests.post(
    f"{BASE_URL}/topics/{fake_topic_id}/resources",
    json=valid_resource
)

if response.status_code == 404:
    error_result = response.json()
    print_json(error_result, "✅ Correct Error for Invalid Topic")
    log_test("Invalid Topic ID Error", True, "Properly rejected")
else:
    log_test("Invalid Topic ID Error", False, f"Got {response.status_code}")


✅ Correct Error for Invalid Type
{
  "error": "Invalid resource_type. Must be one of: ['video', 'image', '3d_model', 'animation', 'virtual_lab', 'pdf', 'interactive', 'article', 'simulation']"
}

✅ PASSED: Invalid Resource Type Error
Details: Properly rejected

✅ Correct Error for Missing Fields
{
  "error": "resource_type, title, and url are required"
}

✅ PASSED: Missing Fields Error
Details: Properly rejected

✅ Correct Error for Invalid Topic
{
  "error": "Topic not found"
}

✅ PASSED: Invalid Topic ID Error
Details: Properly rejected


## Test Summary

In [16]:
# Display test results summary
print("\n" + "="*60)
print("TEST SUMMARY")
print("="*60)

df_results = pd.DataFrame(test_results)
total_tests = len(df_results)
passed_tests = len(df_results[df_results['passed'] == True])
failed_tests = total_tests - passed_tests

print(f"\nTotal Tests: {total_tests}")
print(f"✅ Passed: {passed_tests}")
print(f"❌ Failed: {failed_tests}")
print(f"Success Rate: {(passed_tests/total_tests*100):.1f}%\n")

print("\nDetailed Results:")
print(df_results[['test_name', 'passed', 'details']].to_string(index=False))

# Show failed tests
failed_df = df_results[df_results['passed'] == False]
if len(failed_df) > 0:
    print("\n⚠️ Failed Tests:")
    for idx, row in failed_df.iterrows():
        print(f"  - {row['test_name']}: {row['details']}")
else:
    print("\n🎉 All tests passed!")


TEST SUMMARY

Total Tests: 13
✅ Passed: 13
❌ Failed: 0
Success Rate: 100.0%


Detailed Results:
                  test_name  passed                                           details
Add Video Resource to Topic    True Resource ID: 82e6ac1c-f12a-4206-a4b8-3f2aa08d52ba
      Add 3D Model Resource    True Resource ID: 16322da2-152a-4911-a21d-5bfe7f68cfcb
   Add Virtual Lab Resource    True Resource ID: 87b6acbe-a7aa-47cb-adc1-69566ad33fed
         Bulk Add Resources    True                               Created 4 resources
    Get All Topic Resources    True                                 Found 7 resources
       Filter by Video Type    True                                    Found 1 videos
 Filter by Virtual Lab Type    True                                      Found 1 labs
            Update Resource    True Updated: Introduction to Photosynthesis - UPDATED
      Get Chapter Resources    True                                Total: 7 resources
       Soft Delete Resource    True        

## Resource Statistics

In [17]:
# Get final resource count and statistics
response = requests.get(f"{BASE_URL}/topics/{topic_id}/resources")

if response.status_code == 200:
    final_resources = response.json()
    
    print("\n" + "="*60)
    print("RESOURCE STATISTICS")
    print("="*60)
    print(f"\nTopic: {final_resources['topic_name']}")
    print(f"Total Resources: {final_resources['count']}")
    
    if final_resources['resources']:
        df = pd.DataFrame(final_resources['resources'])
        
        # Count by type
        print("\nResources by Type:")
        type_counts = df['resource_type'].value_counts()
        for rtype, count in type_counts.items():
            print(f"  {rtype}: {count}")
        
        # Show resource list
        print("\n📋 All Resources:")
        print(df[['resource_type', 'title', 'is_active', 'order_index']].to_string(index=False))
else:
    print(f"❌ Could not fetch statistics: {response.status_code}")

ConnectionError: HTTPConnectionPool(host='localhost', port=5200): Max retries exceeded with url: /api/topics/95f24d0b-8723-49ab-80a3-60ff723cb97a/resources (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x117239890>: Failed to establish a new connection: [Errno 61] Connection refused'))

## Cleanup (Optional)

Uncomment to delete all test resources

In [ ]:
# # CLEANUP - Uncomment to delete all test resources
# response = requests.get(f"{BASE_URL}/topics/{topic_id}/resources?is_active=true")
# if response.status_code == 200:
#     resources = response.json()['resources']
#     print(f"Deleting {len(resources)} resources...")
    
#     for resource in resources:
#         del_response = requests.delete(f"{BASE_URL}/resources/{resource['id']}/permanent")
#         if del_response.status_code == 200:
#             print(f"✅ Deleted: {resource['title']}")
#         else:
#             print(f"❌ Failed to delete: {resource['title']}")
    
#     print("\n✅ Cleanup complete!")
# else:
#     print("❌ Could not fetch resources for cleanup")